# pandas：处理表格和混杂数据

## pandas可以让数据清洗和分析工作变得更快更简单。pandas经常和其他工具一同使用，比如numpy和scipy，以及statsmodels和scikit-learn、matplotlib等等。

In [ ]:
import pandas as pd #要使用pandas先引入，简写为pd
from pandas import Series, DataFrame #从pandas引入常用的两个数据结构Series和DataFrame

##  一、pandas 数据结构——Series和DataFrame

### （一）Series

Series类似于一维数组，包括右边的数据value和左边的索引index（0到N-1），可以用series.values和series.index获取索引对象和数组表示形式

In [ ]:
obj = pd.Series([4, 7, -5, 3])
obj.values
obj.index  # like range(4)

In [ ]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c']) #给每一个数据指定一个特殊的索引，比如这里的obj2['a']=-5

In [ ]:
obj2['d'] = 6 #重新给索引d 赋值为6，取代了原本的4，也可以通过给索引赋值修改索引，如obj.index=['Bob','Steve','Jeff','Ryan']
obj2[['c', 'a', 'd']] #也可以通过一个索引表获取一组数据

如果用NumPy函数的话，或者对value进行计算（如过滤、标量乘法等），也都会将索引值保留下来

In [ ]:
obj2[obj2 > 0]
obj2 * 2
np.exp(obj2)

也可以将Series看成一个定长的有序字典

In [ ]:
'b' in obj2 #结果为 True
'e' in obj2 #结果为 False

可以直接通过字典来创建Series，字典的key就是index，字典的value也是Series的value。如果传入一组字典的key的话，输出的Series就会按这组key排序，新key（如California）对应的value为NaN（⾮数字，在pandas中⽤于表示缺失或NA值），而不在states中的Utah不会显示在结果中。

In [ ]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)

In [ ]:
pd.isnull(obj4) #pandas⽤于检测缺失数据的方法， California对应True
pd.notnull(obj4) #pandas⽤于检测缺失数据的方法， California对应False
obj4.isnull() #series中类似的检测缺失数据的方法， California对应 True

Series的索引标签自动对齐功能，哪怕某两个Series的索引顺序不同，但在对这两个数据做运算时，还是会分别对两个Series中的相同索引对应的value做运算，如果某一个index没有在两个Series中同时存在，那运算结果为NaN

In [ ]:
obj3 #索引顺序'Ohio', 'Texas', 'Oregon', 'Utah'
obj4 #索引顺序'California', 'Ohio', 'Oregon', 'Texas'
obj3 + obj4

Series对象本身及其索引都有⼀个name属性

In [ ]:
obj4.name = 'population' #给Series的值命名为人口“population”
obj4.index.name = 'state' #给Series的索引命名为州“state”，那么这个Series就表示每个州对应的人口数

### （二）DataFrame

DataFrame是表格型的数据结构，包括许多列，每一列的值可以有不同的类型；DataFrame有行索引和列索引，它可以被看做由Series组成的字典（共⽤同⼀个索引）。最常⽤的⼀种建DataFrame的办法是直接传⼊⼀个由等⻓列表或NumPy数组组成的字典，结果DataFrame会⾃动加上索引（和Series⼀样），且所有列会被有序排列

In [ ]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

对于特别⼤的DataFrame，head⽅法会选取前五⾏

In [ ]:
frame.head()

如果指定了列序列，那么DataFrame的列就会按照指定的顺序进⾏排列

In [ ]:
pd.DataFrame(data, columns=['year', 'state', 'pop'])

如果传⼊的列在数据中找不到，就会在结果中产⽣缺失值NaN

In [ ]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                      index=['one', 'two', 'three', 'four',
                             'five', 'six'])

通过类似字典标记的⽅式或属性的⽅式，可以将DataFrame的列获取为⼀个Series。frame2[column]适⽤于任何列的名，但是frame2.column只有在列名是⼀个合理的Python变量名时才适⽤

In [ ]:
frame2['state']
frame2.year

⾏也可以通过loc属性获取

In [ ]:
frame2.loc['three']

列可以通过赋值的⽅式进⾏修改。例如，我们可以给那个空的"debt"列赋上⼀个标量值或⼀组值

In [ ]:
frame2['debt'] = 16.5 #赋标量
frame2['debt'] = np.arange(6.) #赋一组值

将列表或数组赋值给某个列时，其⻓度必须跟DataFrame的⻓度相匹配。如果赋值的是⼀个Series，就会精确匹配DataFrame的索引，所有的空位都将被填上缺失值

In [ ]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val

为不存在的列赋值会创建出⼀个新列，关键字del⽤于删除列。

In [ ]:
frame2['eastern'] = frame2.state == 'Ohio'

In [ ]:
del frame2['eastern'] #删除这一列

另一种创建DataFrame的形式——嵌套字典，如果嵌套字典传给DataFrame，那么外层字典的键就会是列，内层键则是⾏索引

In [ ]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}
frame3 = pd.DataFrame(pop)

可以对DataFrame进⾏转置

In [ ]:
frame3.T

### Index Objects索引对象

与python的集合不同，pandas的Index可以包含重复的标签，选择重复的标签，会显示所有的结果

In [ ]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])

## 二、基本功能

### （一）重新索引

reindex将会根据新索引进⾏重排。如果某个索引值当前不存在，就引⼊缺失值

In [ ]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])

对于时间序列这样的有序数据，重新索引时可能需要做⼀些插值处理。method选项即可达到此⽬的，例如，使⽤ffill可以实现前向值填充。

In [ ]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3.reindex(range(6), method='ffill')

借助DataFrame，reindex可以修改（⾏）索引和列。只传递⼀个序列时，会重新索引结果的⾏

In [ ]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame2 = frame.reindex(['a', 'b', 'c', 'd'])

In [ ]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states) #⽤columns关键字重新索引列

### （二）丢弃指定轴上的项

In [ ]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
new_obj = obj.drop('c') #drop⽅法返回的是⼀个在指定轴上删除了指定值的新对象
obj.drop(['d', 'c'])

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four']) #创建DataFrame
data.drop(['Colorado', 'Ohio']) #从⾏标签（axis 0）删除值
data.drop('two', axis=1) #通过传递axis=1或axis='columns'可以删除列的值
data.drop(['two', 'four'], axis='columns') #通过传递axis=1或axis='columns'可以删除列的值

### （三）索引、选取和过滤

#### 1、索引

Series的索引值不只是整数

In [ ]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj['b'] #结果为1.0
obj[1] #结果为1.0
obj[2:4] #结果是c  2.0   d   3.0
obj[['b', 'a', 'd']]
obj[[1, 3]] #结果是b  1.0   d   3.0
obj[obj < 2]

利⽤标签的切⽚运算包含末端

In [ ]:
obj['b':'c']  #结果是b  1.0   c   2.0
obj['b':'c'] = 5 #⽤切⽚可以对Series的相应部分进⾏设置

⽤⼀个值或序列对DataFrame进⾏索引其实就是获取⼀个或多个列

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data['two']
data[['three', 'one']]

In [ ]:
data[:2] #通过切⽚或布尔型数组选取数据
data[data['three'] > 5] #通过切⽚或布尔型数组选取数据

通过布尔型DataFrame进行索引

In [ ]:
data < 5
data[data < 5] = 0
data

#### 2、⽤loc和iloc进⾏选取

In [ ]:
data.loc['Colorado', ['two', 'three']] #通过标签选择⼀⾏和多列

In [ ]:
data.iloc[2, [3, 0, 1]]
data.iloc[2]
data.iloc[[1, 2], [3, 0, 1]]

In [ ]:
data.loc[:'Utah', 'two']
data.iloc[:, :3][data.three > 5]

#### 3、整数索引

为了进⾏统⼀，如果轴索引含有整数，数据选取总会使⽤标签。为了更准确，应使⽤loc（标签）或iloc（整数）

In [ ]:
ser = pd.Series(np.arange(3.))

In [ ]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

In [ ]:
ser[:1] #更准确
ser.loc[:1]
ser.iloc[:1]

### （四）算术运算和数据对齐

pandas可以对不同索引的对象进⾏算术运算。在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集

In [ ]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])

In [ ]:
s1 + s2

对于DataFrame，对⻬操作会同时发⽣在⾏和列上

In [ ]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [ ]:
df1 + df2 #把它们相加后将会返回⼀个新的DataFrame，其索引和列为原来那两个DataFrame的并集。

In [ ]:
df1 = pd.DataFrame({'A': [1, 2]}) #如果DataFrame对象相加，没有共⽤的列或⾏标签，结果都会是空
df2 = pd.DataFrame({'B': [3, 4]})
df1 - df2

### （五）在算术方法中填充值

In [ ]:
df1.add(df2, fill_value=0) #用于加法

In [ ]:
df1.rdiv(1) #用于除法

In [ ]:
df1.reindex(columns=df2.columns, fill_value=0) #对Series或DataFrame重新索引时，也可以指定⼀个填充值

### （六）DataFrame和Series之间的运算

In [ ]:
arr = np.arange(12.).reshape((3, 4))
arr - arr[0] #从arr减去arr[0]，每⼀⾏都会执⾏这个操作。这就叫做⼴播（broadcasting）

In [ ]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.iloc[0] 
frame - series #默认情况下，DataFrame和Series之间的算术运算会将Series的索引匹配到DataFrame的列，然后沿着⾏⼀直向下⼴播

如果某个索引值在DataFrame的列或Series的索引中找不到，则参与运算的两个对象就会被重新索引以形成并集

In [ ]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame + series2

### （七）函数应用和映射

In [ ]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
np.abs(frame)#用NumPy的ufuncs（元素级数组⽅法）操作pandas对象

In [ ]:
f = lambda x: x.max() - x.min()
frame.apply(f) #将函数应⽤到由各列或⾏所形成的⼀维数组上

In [ ]:
frame.apply(f, axis='columns') #传递axis='columns'到apply，这个函数在每⾏执⾏

In [ ]:
format = lambda x: '%.2f' % x #想得到frame中各个浮点值的格式化字符串
frame.applymap(format)

### （八）排序和排名

In [ ]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index() #使⽤sort_index⽅法，它会返回⼀个已排序的新对象

In [ ]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)), #DataFrame，则可以根据任意⼀个轴上的索引进⾏排序
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])
frame.sort_index()
frame.sort_index(axis=1)

In [ ]:
frame.sort_index(axis=1, ascending=False) #ascending=False表示降序

In [ ]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values() #按值对Series进⾏排序

In [ ]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values() #排序时，任何缺失值默认都会被放到Series的末尾

In [ ]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame.sort_values(by='b') #根据⼀个或多个列中的值进⾏排序，只需要将⼀个或多个列的名字传递给sort_values的by选项即可

In [ ]:
frame.sort_values(by=['a', 'b']) #根据多个列进⾏排序

In [ ]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank() #rank是通过“为各组分配⼀个平均排名”的⽅式破坏平级关系的

In [ ]:
obj.rank(method='first') #根据值在原数据中出现的顺序给出排名

### （九）带有重复标签的轴索引

In [ ]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj.index.is_unique #索引的is_unique属性可以告诉你它的值是否是唯⼀的

如果某个索引对应多个值，则返回⼀个Series；⽽对应单个值的，则返回⼀个标量值：

In [ ]:
obj['a']
obj['c']

## 三、汇总和计算描述统计

### （一）汇总

In [ ]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])

In [ ]:
df.sum() #sum⽅法将会返回⼀个含有列的和的Series

In [ ]:
df.sum(axis='columns') #传⼊axis='columns'或axis=1将会按⾏进⾏求和运算

In [ ]:
df.mean(axis='columns', skipna=False) #NA值会⾃动被排除，除⾮整个切⽚（这⾥指的是⾏或列）都是NA。通过skipna选项可以禁⽤该功能

In [ ]:
df.idxmax() #间接统计（⽐如达到最⼤值的索引）

In [ ]:
df.cumsum() #累计型

In [ ]:
df.describe() #⽤于⼀次性产⽣多个汇总统计

### （二）相关系数与协⽅差

In [ ]:
conda install pandas-datareader #安装

In [ ]:
returns['MSFT'].corr(returns['IBM']) #Series的corr⽅法⽤于计算两个Series中重叠的、⾮NA的、按索引对⻬的值的相关系数。与此类似，cov⽤于计算协⽅差
returns['MSFT'].cov(returns['IBM'])

In [ ]:
returns.MSFT.corr(returns.IBM) #更简洁的语法选择列

In [ ]:
returns.corr() #DataFrame的corr和cov⽅法将以DataFrame的形式分别返回完整的相关系数或协⽅差矩阵
returns.cov()

In [ ]:
returns.corrwith(returns.IBM) #利⽤DataFrame的corrwith⽅法，你可以计算其列或⾏跟另⼀个Series或DataFrame之间的相关系数。传⼊⼀个Series将会返回⼀个相关系数值Series（针对各列进⾏计算）

In [ ]:
returns.corrwith(volume) #传⼊⼀个DataFrame则会计算按列名配对的相关系数

### （三）唯⼀值、值计数以及成员资格

In [ ]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

In [ ]:
uniques = obj.unique() #unique，它可以得到Series中的唯⼀值数组，返回的唯⼀值是未排序的，如果需要的话，可以对结果再次进⾏排序（uniques.sort()）

In [ ]:
obj.value_counts() #计算⼀个Series中各值出现的频率

In [ ]:
pd.value_counts(obj.values, sort=False) #value_counts可⽤于任何数组或序列

In [ ]:
mask = obj.isin(['b', 'c']) #isin⽤于判断⽮量化集合的成员资格，可⽤于过滤Series中或DataFrame列中数据的⼦集
obj[mask]

In [ ]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c', 'b', 'a'])
pd.Index(unique_vals).get_indexer(to_match) #是Index.get_indexer⽅法，它可以给你⼀个索引数组，从可能包含重复值的数组到另⼀个不同值的数组

In [ ]:
data = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
                     'Qu2': [2, 3, 1, 2, 3],
                     'Qu3': [1, 5, 2, 4, 4]})
result = data.apply(pd.value_counts).fillna(0) #结果中的⾏标签是所有列的唯⼀值。这些值是每个列中这些值的相应计数。